In [ ]:
import time  
import requests
import json
import re
import random
import urllib.request as request
from hashlib import md5

In [ ]:
def writeLog(text):
    fh = open('Log.txt', 'a')
    fh.write(text+'\n')
    fh.close()

In [ ]:
# 切换IP（拨号上网）
def changeIP(ADSLusername, ADSLpassword):
    # 断开宽带连接
    cmdDisconnect ="rasdial 宽带连接 /disconnect"
    os.system(cmdDisconnect)
    time.sleep(2)                  
    # 拨号连接
    cmdConnect = "rasdial 宽带连接 %s %s"  %(ADSLusername, ADSLpassword)
    os.system(cmdConnect)

In [ ]:
# 获取外网当前IP
def getIP():
    url = r'http://2018.ip138.com/ic.asp'
    redata = requests.get(url).text
    ip = re.findall("\[(.*)\]", redata)[0]
    return ip

In [ ]:
# 若快打码，用于验证码识别
class RClient(object):
    def __init__(self, username, password, soft_id, soft_key):
        self.username = username
        self.password = md5(password.encode("utf8")).hexdigest()
        self.soft_id = soft_id
        self.soft_key = soft_key
        self.base_params = {
            'username': self.username,
            'password': self.password,
            'softid': self.soft_id,
            'softkey': self.soft_key,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'Expect': '100-continue',
            'User-Agent': 'ben',
        }
    def rk_create(self, im, im_type, timeout=60):
        """
        im: 图片字节
        im_type: 题目类型
        """
        params = {
            'typeid': im_type,
            'timeout': timeout,
        }
        params.update(self.base_params)
        files = {'image': ('a.jpg', im)}
        r = requests.post('http://api.ruokuai.com/create.json', data=params, files=files, headers=self.headers)
        return r.json()
    def rk_report_error(self, im_id):
        """
        im_id:报错题目的ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://api.ruokuai.com/reporterror.json', data=params, headers=self.headers)
        return r.json()

In [ ]:
# 号码短租，用于注册用手机号的获取
def dzLogin(session, dzAccount, dzPassword):
    logindata = session.get('http://api.jmyzm.com/http.do?action=loginIn&uid='+str(dzAccount)+'&pwd='+str(dzPassword)).content.decode(encoding='UTF8')
    if len(re.findall(dzAccount+r'\|(.*)',logindata))>0:
        dzLoginToken = re.findall(dzAccount+r'\|(.*)',logindata)[0]
        print("短租登陆成功!")
        writeLog("短租登陆成功!")    
    elif logindata=='login_error':
        dzLoginToken = 0
        print("短租账号密码有误!")
        writeLog("短租账号密码有误!")
    elif logindata=='account_is_closed':
        dzLoginToken = 0
        print("短租账号被关闭（登录官网进入安全中心开启!）")
        writeLog("短租账号被关闭（登录官网进入安全中心开启!）")
    elif logindata=='account_is_stoped':
        dzLoginToken = 0
        print("短租账号被停用!")
        writeLog("短租账号被停用!")
    elif logindata=='account_is_locked':
        dzLoginToken = 0
        print("短租账号被锁定（无法取号，充值任意金额解锁，请登录官网查看详情!）")
        writeLog("短租账号被锁定（无法取号，充值任意金额解锁，请登录官网查看详情!）")
    else:
        dzLoginToken = 0
        print("其它错误!")
        writeLog("其它错误!")
    return dzLoginToken

In [ ]:
# 获取手机号码
def getMobilenum(session, dzAccount, dzLoginToken):
    code = 1
    count = 0
    while(code==1 and count<3):
        count+=1
        if count>1:
            print("手机号码获取失败，尝试第"+str(count)+"次")
            writeLog("手机号码获取失败，尝试第"+str(count)+"次")
        getMobilenumData = session.get('http://api.jmyzm.com/http.do?action=getMobilenum&pid=99&uid='+dzAccount+'&token='+dzLoginToken+'&mobile=&size=1').content.decode(encoding='UTF8')
        if len(re.findall(r'\|(.*)',getMobilenumData))>0:
            if re.findall(r'\|(.*)',getMobilenumData)[0]=='to_fast_try_again ':
    #             print("手机号码获取速度过快，稍后重试")
                time.sleep(3)
            else:
                mobileNum = re.findall(r'(.*)\|',getMobilenumData)[0]
                mobileToken = re.findall(r'\|(.*)',getMobilenumData)[0]
                code = 0
                print("获取手机号码 "+str(mobileNum))
                writeLog("获取手机号码 "+str(mobileNum))
                return [mobileNum, mobileToken]
    return code

In [ ]:
# 检查获取到的手机号码是否无效
def testNumber(session, account):
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/reg/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    Time = str(int(time.time() * 1000))
    data = {'op': 'checkmobile', 
            'mobile': str(account), 
            'r': '0.5977777634092578', 
            'quc_lang': '', 
            'callback': 'QiUserJsonP'+Time
           }
    redata = session.get('http://login.360.cn/index.php?op=checkmobile&mobile='+str(account)+'&r=0.5977777634092578&quc_lang=&callback=QiUserJsonP'+Time, data=data, headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if int(redatadict.get('errno'))==0:
        print("手机号可以使用！下一步")
        writeLog("手机号可以使用！下一步")
        return 0
    elif int(redatadict.get('errno'))==30000:
        print("手机已被使用，切换手机号")
        writeLog("手机已被使用，切换手机号")
        return 1
    else:
        print("其它错误")
        writeLog("其它错误")
        return 2

In [ ]:
# 获取短信验证码
def recSmsCode(session, dzAccount, mobileNum, mobileToken):
    code = 1
    count = 0
    time.sleep(5)
    while(code==1 and count<10):
        count+=1
        if count>1:
            print("短信验证码获取失败，尝试第"+str(count)+"次")
            writeLog("短信验证码获取失败，尝试第"+str(count)+"次")
            time.sleep(count*2)
        getSmsCodeData = session.get('http://api.jmyzm.com/http.do?action=getVcodeAndReleaseMobile&uid='+str(dzAccount)+'&token='+mobileToken+'&mobile='+str(mobileNum)).content.decode(encoding='UTF8')
        if len(re.findall(r'\|(.*)',getSmsCodeData))>0:
            if re.findall(r'\|(.*)',getSmsCodeData)[0]=='to_fast_try_again ':
    #             print("短信获取速度过快，稍后重试")
                time.sleep(3)
            else:
                smsCode = re.findall(r'：(.*)，用', getSmsCodeData)[0]
                code = 0
                print("短信验证码为 "+str(smsCode))
                writeLog("短信验证码为 "+str(smsCode))
                return smsCode
        elif getSmsCodeData=='not_receive':
            time.sleep(5)
    # 加黑多次获取不到短信的手机号码
    if code==1:
        session.get('http://api.jmyzm.com/http.do?action=addIgnoreList&uid='+str(dzAccount)+'&token='+mobileToken+'&mobile='+str(mobileNum)+'&pid=99')
    return code

In [ ]:
# 获取注册用图形验证码，并识别返回
def getCode(session, rkAccount, rkPassword): 
    headers = {'User-Agent':'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/reg/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F'}
    Time = str(int(time.time() * 1000))
    data = {'src': 'pcw_i360',
            'o': 'sso',
            'm': 'getCaptcha',
            'app': 'i360',
            'scene': 'strictreg',
            'callback': 'QiUserJsonP'+Time
           }
    codedata = session.get('http://login.360.cn/?src=pcw_i360&o=sso&m=getCaptcha&app=i360&scene=strictreg&callback=QiUserJsonP'+Time, data=data, headers=headers).content.decode(encoding='UTF8')
    codedatadict = json.loads(re.findall(r"\((.*)\)", codedata)[0])
    codeToken = codedatadict.get('crypt_code')
    codeImgUrl = codedatadict.get('img')
    request.urlretrieve(codeImgUrl, './code.jpg')
    rc = RClient(rkAccount, rkPassword, '106432', 'b46894ebdead4d4eb3c6703b58da5b24')
    im = open('./code.jpg', 'rb').read()
    codeResult = rc.rk_create(im, 3000)['Result']
    print("识别验证码为 "+codeResult)
    writeLog("识别验证码为 "+codeResult)
    return codeToken, codeResult

In [ ]:
# 获取注册用短信验证码（触发）
def getSmsCode(session, account, codeResult, codeToken, count):
    count+=1
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/reg/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    Time = str(int(time.time() * 1000))
    data = {'src': 'undefined', 
            'o': 'User', 
            'm': 'sendSmsCodeNew', 
            'account': account, 
            'captcha': codeResult, 
            'vt': '', 
            'condition': '2', 
            'crumb': '',
            'sms_scene': '2',
            'iswap': '1',
            'requestScema': 'http', 
            'quc_lang': '', 
            'qucrypt_code': codeToken,
            'callback': 'QiUserJsonP'+Time
           }
    redata = session.get('http://login.360.cn/?src=undefined&o=User&m=sendSmsCodeNew&account='+str(account)+'&captcha='+str(codeResult)+'&vt=&condition=2&crumb=&sms_scene=2&iswap=1&requestScema=http&quc_lang=&qucrypt_code='+codeToken+'&callback=QiUserJsonP'+Time, data=data, headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if int(redatadict.get('errno'))==0:
        print("短信验证码已发送！")
        writeLog("短信验证码已发送！")
    elif int(redatadict.get('errno'))==78000 and count<3:
        print("图形验证码错误，尝试第"+str(count+1)+"次")
        writeLog("图形验证码错误，尝试第"+str(count+1)+"次")
        codeToken, codeResult = getCode(session, rkAccount, rkPassword)
        getSmsResult = getSmsCode(session, account, codeResult, codeToken, count)
    else:
        print("短信验证码发送失败")
        writeLog("短信验证码发送失败")

In [ ]:
# 注册主函数
def register(session, account, password, ip, codeResult, codeToken, smsCode):
    headers = {'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
               'Accept-Language' : 'zh-CN,zh;q=0.9',
               'Cache-Control': 'max-age=0', 
               'Connection': 'keep-alive',
               'Content-Length': '545', 
               'Content-Type': 'application/x-www-form-urlencoded', 
               'Host': 'i.360.cn',
               'Origin': 'http://i.360.cn', 
               'Referer': 'http://i.360.cn/reg/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F',
               'Upgrade-Insecure-Requests': '1',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    HD5password  = md5(password.encode("utf8")).hexdigest()
    Time = str(int(time.time() * 1000))
    data = {'callback': '_CrossDomainCallback'+Time, 
            'proxy': '//i.360.cn/psp_jump.html', 
            'account': account, 
            'acctype': '2', 
            'password': HD5password, 
            'rePassword': HD5password, 
            'captcha': codeResult, 
            'smscode': smsCode, 
            'src': 'mpw_blockcat', 
            'is_agree': '1', 
            'charset': 'undefined', 
            'loginEmailActiveFlag': '0', 
            'captchaApp': 'i360', 
            'qucrypt_code': codeToken, 
            'captchaFlag': '1', 
            'proxy': 'http://i.360.cn/psp_jump.html', 
            'callback': 'parent.QHPass.regUtils.submitCB', 
            'requestScema': 'http', 
            'quc_lang': ''
           }
    redata = session.post('http://i.360.cn/reg/doregAccount', data=data, headers=headers).content.decode(encoding='UTF8')
    if len(re.findall(r'&errno=(.*)&errmsg=',redata))>0:
        if int(re.findall(r'&errno=(.*)&errmsg=',redata)[0])==0:
            print("注册成功！")
            writeLog("注册成功！")
            fh = open('注册成功.txt', 'a')
#             fh.write(account+"----"+password+"\n")
            fh.write(str(account)+"----"+password+"----"+str(ip)+"----"+str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime()))+"\n")
            fh.close()
        else:
            print("注册失败")
            writeLog("注册失败")
    else:
        print("注册失败")
        writeLog("注册失败")

In [ ]:
# 获取领猫用图形验证码，并识别返回
def getAwardCode(session, rkAccount, rkPassword):
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    data = {'__did': '', 
            '__src': '', 
            'callback': 'helpers.JsonpReq.completeCall.call_1'
           }
    redata = session.get('https://api.blockcat.360.cn/captcha/img/getGameCode?__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', data=data, headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    codeImgUrl =  redatadict.get('data')
    request.urlretrieve(codeImgUrl, './code.jpg')
    rc = RClient(rkAccount, rkPassword, '106432', 'b46894ebdead4d4eb3c6703b58da5b24')
    im = open('./code.jpg', 'rb').read()
    codeResult = rc.rk_create(im, 3000)['Result']
    return codeResult

In [ ]:
# 获取领猫用验证码（触发）
def getGameSmsCode(session, rkAccount, rkPassword, count):
    count+=1
    codeResult = getAwardCode(session, rkAccount, rkPassword)
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    redata = session.get('https://api.blockcat.360.cn/captcha/sms/getGameCode?img_code='+codeResult+'&__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if int(redatadict.get('errno'))==0:
        print("短信验证码已发送！")
        writeLog("短信验证码已发送！")
    elif int(redatadict.get('errno'))==78000 and count<3:
        print("图形验证码错误，尝试第"+str(count+1)+"次")
        writeLog("图形验证码错误，尝试第"+str(count+1)+"次")
        getGameSmsCode(session, codeResult, count)
    else:
        print("短信验证码发送失败")
        writeLog("短信验证码发送失败")

In [ ]:
# 获取领猫用Token
def getAwardToken(session):
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    data = {'__did': '', 
            '__src': '', 
            'callback': 'helpers.JsonpReq.completeCall.call_1'
           }
    redata = session.get('https://api.blockcat.360.cn/kitty/award/info?__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', data=data, headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if int(redatadict.get('errno'))==0:
        return redatadict.get('data').get('phone_sec')
    else:
        return 1

In [ ]:
# 领取猫咪
def getKitty(code, awardToken, smsCode):
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    data = {'phone_sec': awardToken, 
            'code': smsCode,
            '__did': '', 
            '__src': '', 
            'callback': 'helpers.JsonpReq.completeCall.call_1'}
    redata = session.get('https://api.blockcat.360.cn/kitty/award/do?phone_sec='+str(awardToken)+'&code='+str(smsCode)+'&__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', data=data, headers=headers).content.decode(encoding='UTF8')
    redatadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if int(redatadict.get('errno'))==0:
        print("猫咪领取成功！")
        writeLog("猫咪领取成功！")
        code=0
    return code

In [ ]:
# 领取猫咪主函数
def getAward(session, rkAccount, rkPassword, phsession, dzAccount, account, mobileToken):
    code = 1
    count = 0
    while(code==1 and count<3):
        if count>0:
            print("猫咪领取失败，尝试第"+str(count+1)+"次")
            writeLog("猫咪领取失败，尝试第"+str(count+1)+"次")
        count+=1
        awardToken = getAwardToken(session)
        if awardToken!=1:
            getGameSmsCode(session, rkAccount, rkPassword, count=0)
            smsCode = recSmsCode(phsession, dzAccount, account, mobileToken)
            code = getKitty(code, awardToken, smsCode)

In [ ]:
if __name__ == "__main__":
    # 第一次使用需要输入1，进行配置
    load = input("是否加载原有配置，是请输入“0”，否请输入“1”：")
    if int(load)==0:
        fh = open('config.txt','r')
        configList = []
        for line in fh.readlines():
            linestr = line.strip()
            listlinestr = linestr.split('：')
            configList.append(listlinestr[1])
        dzAccount = configList[0]
        dzPassword = configList[1]
        rkAccount = configList[2]
        rkPassword = configList[3]
        isChangeIP = configList[4]
        if int(isChangeIP)==0:
            ADSLusername = configList[5]
            ADSLpassword = configList[6]
        fh.close()
    else:
        dzAccount = input("请输入号码短租账号：")
        dzPassword = input("密码：")
        rkAccount = input("请输入若快账号：")
        rkPassword = input("密码：")
        isChangeIP = input("是否开启拨号切换IP功能，是请输入“0”，否请输入“1”：")
        fh = open('config.txt', 'w')
        fh.write("号码短租账号："+str(dzAccount)+"\n号码短租密码："+str(dzPassword)+"\n若快账号："+str(rkAccount)+
                 "\n若快密码："+str(dzPassword)+"\n是否自动拨号："+str(isChangeIP)+"\n")
        if int(isChangeIP)==0:
            ADSLusername = input("请输入拨号上网账号：")
            ADSLpassword = input("密码：")
            fh.write("拨号上网账号："+str(ADSLusername)+"\n"+"拨号上网密码："+str(ADSLpassword)+"\n")
        fh.close()
    code = 1
    smsWrongCode = 0
    registerCount = 0
    counter = 0
    amount = input("要注册账号数量：")
    ip = getIP()
    with requests.session() as phsession:
        dzLoginToken = dzLogin(phsession, dzAccount, dzPassword)
        if dzLoginToken!=0:
            while(code==1 and registerCount<int(amount)):
                print("-------------------------------------------\nTime:"+
                      str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                writeLog("-------------------------------------------\nTime:"+
                      str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
                # 上一轮注册成功则切换IP
                if registerCount>counter:
                    counter=registerCount
                    changeIP(ADSLusername, ADSLpassword)
                    ip = getIP()
                mobileNumResult = getMobilenum(phsession, dzAccount, dzLoginToken)
                if mobileNumResult==1:
                    print("\n\n手机号码多次获取失败，程序停止!")
                    writeLog("\n\n手机号码多次获取失败，程序停止!")
                    code = 0
                else:
                    account = mobileNumResult[0]
                    mobileToken = mobileNumResult[1]
                    password = ''.join(random.sample('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789',10))
                    with requests.session() as session:
                        atcode = testNumber(session, account)
                        if atcode==0:
                            codeToken, codeResult = getCode(session, rkAccount, rkPassword)
                            getSmsResult = getSmsCode(session, account, codeResult, codeToken, count=0)
                            smsCode = recSmsCode(phsession, dzAccount, account, mobileToken)
                            if smsCode==1:
                                smsWrongCode+=1
                                if smsWrongCode>4:
                                    print("\n\n短信多次获取失败，程序停止!")
                                    writeLog("\n\n短信多次获取失败，程序停止!")
                                    code = 0
                            else:
                                smsWrongCode = 0
                                register(session, account, password, ip, codeResult, codeToken, smsCode)
                                registerCount+=1
                                getAward(session, rkAccount, rkPassword, phsession, dzAccount, account, mobileToken)
                        else:
                            # 加黑无用手机号码
                            phsession.get('http://api.jmyzm.com/http.do?action=addIgnoreList&uid='+str(dzAccount)+'&token='+mobileToken+'&mobile='+str(account)+'&pid=99')